In [1]:
import pandas as pd
import numpy as np
import random
from autocorrect import Speller
import itertools
import re
from LLM import LLM
from PromptBuilder import PromptBuilder
from metadata import language_registers, discussion_tones

2025-04-05 21:28:22.755285: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-05 21:28:22.765718: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743881302.778908 3992539 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743881302.782975 3992539 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-05 21:28:22.796682: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
a = pd.read_excel('PRO-CTCAE_Questionnaire_Terminology.xls', sheet_name = 'PRO')
index = (a [a['PRO-CTCAE PT'] == 'Pain and swelling at injection site']).index.values[0] +1 
symptoms = a.iloc[:index]['PRO-CTCAE PT'].values

Dict = {}

for symptom in symptoms : 

    try:
        Dict[symptom] = {}
        descriptions_code = a[a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute Code'].values[0].split(" || ")
        descriptions = a [a['PRO-CTCAE PT'] == symptom]['Has PRO-CTCAE Attribute PT'].values[0].split(" || ")
        for description in descriptions :
            Dict[symptom][description]  = a[a['PRO-CTCAE PT'] == description  ]['PRO-CTCAE Value PT'].values[0].split(" || ")
    
    except Exception as e :
        print(e)

'float' object has no attribute 'split'


In [3]:
for el in Dict.keys():
    for el2 in Dict[el].keys():
        if "Not sexually active" in Dict[el][el2]:
            Dict[el][el2].remove("Not sexually active")

del Dict['Other Symptoms']

print("The dictionary has been cleaned and is ready to be used")

The dictionary has been cleaned and is ready to be used


In [4]:
model = LLM("iRASC/BioLlama-Ko-8B")

/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:862: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/laajila/miniconda3/envs/darts_env/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Device set to use cuda:0


Model loaded on device(s): {'': 0}


In [6]:
Dict['Abdominal pain']

{'Interference (with daily activities)': ['Not at all',
  'A little bit',
  'Somewhat',
  'Quite a bit',
  'Very much'],
 'Frequency': ['Almost constantly',
  'Prefer not to answer',
  'Occasionally',
  'Frequently',
  'Never',
  'Rarely'],
 'Severity': ['Very severe',
  'Prefer not to answer',
  'None',
  'Not applicable',
  'Moderate',
  'Mild',
  'Severe']}

## Generate one symptom per sentence 

In [ ]:
import itertools

prompt_builder = PromptBuilder()

data = []
c = 0
for symptom, descriptions in Dict.items():

    # Generate all (description, meta) combinations
    description_meta_combinations = list(itertools.product(descriptions.keys(), *descriptions.values()))

    for description, *meta_combinations in description_meta_combinations:

        for meta_set in meta_combinations:

            if c % 100 == 0:
                print(f"Generating {c+100} sentences...")
        
            detail_level = np.random.choice([1, 2, 3, 4, 5])
                        
            enumeration = np.random.choice([True, False], p=[0.2, 0.8])

            explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])


            language_style = random.choice(language_registers)['name']
            tone = random.choice(discussion_tones)['name']
            spelling_errors = random.choice([True, False])

            prompt = prompt_builder.build_prompt(
                symptoms=[symptom],
                description=description,
                meta=meta_set,
                detail_level=detail_level,
                enumeration=enumeration,
                explicit_symptom=explicit_symptom,
                language_style=language_style,
                spelling_errors=spelling_errors,
                tone=tone
            )
            
            phrase_generated = model.generate_text(messages=prompt)

            data.append([
                phrase_generated, symptom, description, meta_set, language_style, 
                tone, detail_level, enumeration, explicit_symptom, spelling_errors
            ])
            c += 1

            df = pd.DataFrame(data, columns=[
                "Dialogue_Generated", "Symptom", "Description", "Meta", 
                "Language_Style", "Tone", "Detail_Level", "Enumeration", 
                "Explicit_Symptom", "Spelling_Errors"
            ])

            df.to_csv("Final_dataset_generated_one_symptom.csv")

Generating 100 sentences...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Generating 200 sentences...


In [8]:
prompt_builder = PromptBuilder()

data = []
c = 0

symptom_list = list(Dict.keys())

# Define how many symptoms per sentence (e.g., 2 to 4 symptoms)
min_symptoms = 2
max_symptoms = 4

for _ in range(100):  # replace this with your desired total number of examples

    # Randomly sample symptoms
    selected_symptoms = random.sample(symptom_list, k=random.randint(min_symptoms, max_symptoms))

    descriptions = []
    metas = []

    # Get description + meta info for each selected symptom
    for symptom in selected_symptoms:

        descriptions_dict = Dict[symptom]

        description = random.choice(list(descriptions_dict.keys()))

        meta_options = descriptions_dict[description]

        meta = random.choice(meta_options)
        
        descriptions.append((symptom, description))
        metas.append(meta)

    if c % 100 == 0:
        print(f"Generated {c+100} sentences...")

    # Parameters shared across the sentence

    detail_level = np.random.choice([1, 2, 3, 4, 5])
    enumeration = np.random.choice([True, False], p=[0.2, 0.8])
    explicit_symptom = np.random.choice([True, False], p=[0.2, 0.8])
    language_style = random.choice(language_registers)['name']
    tone = random.choice(discussion_tones)['name']
    spelling_errors = random.choice([True, False])

    # Build the prompt using all symptoms
    symptoms_only = [s[0] for s in descriptions]
    descriptions_only = [s[1] for s in descriptions]
    
    prompt = prompt_builder.build_prompt(
        symptoms=symptoms_only,
        description=descriptions_only,  # assumes you support a list of descriptions
        meta=metas,
        detail_level=detail_level,
        enumeration=enumeration,
        explicit_symptom=explicit_symptom,
        language_style=language_style,
        spelling_errors=spelling_errors,
        tone=tone
    )

    phrase_generated = model.generate_text(messages=prompt)

    data.append([
        phrase_generated, symptoms_only, descriptions_only, metas, 
        language_style, tone, detail_level, enumeration, 
        explicit_symptom, spelling_errors
    ])
    c += 1

df = pd.DataFrame(data, columns=[
    "Dialogue_Generated", "Symptoms", "Descriptions", "Metas", 
    "Language_Style", "Tone", "Detail_Level", "Enumeration", 
    "Explicit_Symptom", "Spelling_Errors"
])

Generated 100 sentences...


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [14]:
df.to_csv('New_generated_multi_symptoms_dataset.csv')